<table>
    <tr>
        <td><img src="./imagenes/Macc.png" width="400"/></td>
        <td>&nbsp;</td>
        <td>
            <h1 style="color:blue;text-align:left">Inteligencia Artificial</h1></td>
        <td>
            <table><tr>
            <tp><p style="font-size:150%;text-align:center">Notebook</p></tp>
            <tp><p style="font-size:150%;text-align:center">Cálculo de eventos</p></tp>
            </tr></table>
        </td>
    </tr>
</table>

---


## Objetivo <a class="anchor" id="inicio"></a>

En este notebook veremos una manera de implementar el razonamiento de sentido común usando el Cálculo de Eventos Discreto.

Basado en Mueller, E. T. (2006). *Commonsense Reasoning*. Morgan Kaufmann/Elsevier, San Francisco. Ver el [sitio web](https://decreasoner.sourceforge.net/) del programa ya implementado.

## Secciones

Desarrollaremos la explicación en las siguientes secciones:

* [Constantes y predicados.](#consts)
* [Fluentes, eventos e instantes.](#fluev)
* [Situaciones.](#sits)
* [Fórmulas.](#forms)
* [Cuantificadores.](#cuants)
* [Axiomas.](#axs)
* [Uso de bases de conocimiento.](#base)
* [Codificación de fórmulas fundamentadas.](#cod)
* [Uso de DPLL.](#dpll)

In [1]:
from logica import *
from ec import *
import pycosat

# Constantes y predicados 
<a class="anchor" id="consts"></a>

([Volver al inicio](#inicio))



**Constantes:**

In [2]:
p = Constante(tipo='agente', nombre='pedro')
print(f'nombre:{p}, tipo:{p.tipo}, clase:{obtener_type(p)}')

nombre:pedro, tipo:agente, clase:Constante


In [3]:
b = Constante(tipo='lugar', nombre='bogota')
print(f'nombre:{b}, tipo:{b.tipo}, clase:{obtener_type(b)}')

nombre:bogota, tipo:lugar, clase:Constante


**Predicados:**

In [4]:
atomo = Atomo(nombre='dormido', tipos_argumentos=['agente'], argumentos=[p])
print(f'str:{atomo}, nombre:{atomo.nombre}, \ntipos:{atomo.predicado.tipos_argumentos}, argumentos:{atomo.argumentos}, clase:{obtener_type(atomo)}')

str:Dormido(pedro), nombre:Dormido, 
tipos:['agente'], argumentos:[<ec.Constante object at 0x7f9d444e8eb0>], clase:Atomo


# Fluentes, eventos e instantes 
<a class="anchor" id="fluev"></a>

([Volver al inicio](#inicio))



**Fluentes:**

In [5]:
flu = Fluente(nombre=atomo.nombre, atomo=atomo)
print(f'str:{flu}, nombre:{flu.nombre}, tipo:{flu.tipo}, clase:{obtener_type(flu)}')

str:f_0, nombre:Dormido, tipo:fluente, clase:Fluente


**Eventos:**

In [6]:
e = Evento(nombre='DESPERTAR', sujeto=p, lugar=b)
print(f'str:{e}, nombre:{e.nombre}, tipo:{e.tipo}, clase:{obtener_type(e)}')
print(e.formular())

str:e_0, nombre:DESPERTAR, tipo:evento, clase:Evento
['DESPERTAR(e_0)', 'SUJETO(e_0,pedro)', 'EN(e_0,bogota)']


**Instantes:**

In [7]:
t = Instante(0)
print(f'valor:{t.valor}, tipo:{t.tipo}, clase:{obtener_type(t)}')

valor:0, tipo:instante, clase:Instante


# Situaciones <a class="anchor" id="sits"></a>

([Volver al inicio](#inicio))

In [8]:
sit = Situacion()
sit.nueva_entidad('objeto', 'luz')
o = sit.entidades['objeto'][0]
for t in range(3):
    sit.nuevo_instante()
sit.nuevo_evento('encender')
sit.nuevo_evento('apagar')
atomo = Atomo('encendida', ['objeto'], [o])
sit.nuevo_fluente(atomo)
print(sit)

Instantes: ['0', '1', '2']

Entidades:
	Tipo: objeto
	luz

Eventos:
	e_0:
	ENCENDER(e_0)

	e_1:
	APAGAR(e_1)

Fluentes:
	f_0: Encendida(luz)



<a class="anchor" id="ej1"></a>**Ejercicio 1:** 

([Próximo ejercicio](#ej2))

Cree una nueva situación `sit2` e incluya las entidades Pedro y Bogotá, el fluente `Dormido` y el evento `DESPERTAR` (con las características mostradas anteriormente), con dos instantes de tiempo. Visualice la nueva situación.

---

# Fórmulas <a class="anchor" id="forms"></a>

([Volver al inicio](#inicio))

Tenemos fórmulas referentes a la relación entre las constantes de eventos, fluentes e instantes:

* Happens(e,t)
* HoldsAt(f,t)
* Initiates(e,f,t)
* Terminates(e,f,t)
* Antes(t1,t2)

Todas estas fórmulas reciben o bien índices o bien constantes:

In [9]:
haps1 = Happens(0, 0)
print(haps1)
holds1 = HoldsAt(0, 0)
print(holds1)
ini1 = Initiates(0, 0, 0)
print(ini1)
term1 = Terminates(0, 0, 0)
print(term1)
rel1 = Antes(0, 1)
print(rel1)

Happens(ev0,ti0)
HoldsAt(flu0,ti0)
Initiates(ev0,flu0,ti0)
Terminates(ev0,flu0,ti0)
ti0<ti1


In [10]:
p = Constante(tipo='agente', nombre='pedro')
b = Constante(tipo='lugar', nombre='bogota')
atomo = Atomo(nombre='dormido', tipos_argumentos=['agente'], argumentos=[p])
flu = Fluente(nombre=atomo.nombre, atomo=atomo)
e = Evento(nombre='DESPERTAR', sujeto=p, lugar=b)
t = Instante(0)
haps = Happens(0, 0, e=e,t=t)
print(haps)

Happens(e_0,0)


**Función** `sust`:

In [11]:
p = Constante(tipo='agente', nombre='pedro')
b = Constante(tipo='lugar', nombre='bogota')
atomo = Atomo(nombre='dormido', tipos_argumentos=['agente'], argumentos=[p])
flu = Fluente(nombre=atomo.nombre, atomo=atomo)
e = Evento(nombre='DESPERTAR', sujeto=p, lugar=b)
t = Instante(0)
haps2 = sust(haps1, 0, e)
print(haps1, '=>', haps2)
term2 = sust(term1, 0, e)
term2 = sust(term2, 0, flu)
term2 = sust(term2, 0, t)
print(term1, '=>', term2)
rel2 = sust(rel1, 1, t)
print(rel1, '=>', rel2)

Happens(ev0,ti0) => Happens(e_0,ti0)
Terminates(ev0,flu0,ti0) => Terminates(e_0,f_0,0)
ti0<ti1 => ti0<0


También tenemos fórmulas relacionadas con las constantes lógicas:

* Y
* O
* Regla
* Negacion

In [12]:
f1 = Y([Happens(0, 1), Antes(0, 1),])
print(f1)
f2 = Negacion(f1)
print(f2)
f3 = Regla(f2, Negacion(Antes(0, 1)))
print(f3)

(Happens(ev0,ti1)∧ti0<ti1)
-(Happens(ev0,ti1)∧ti0<ti1)
(-(Happens(ev0,ti1)∧ti0<ti1)⇒-ti0<ti1)


# Cuantificadores <a class="anchor" id="cuants"></a>

([Volver al inicio](#inicio))

**Cuantificador universal:**

In [13]:
e = sit.eventos[0]
f = sit.fluentes[0]
Qt = Cuantificador('instante', 'todo', 0)
form1 = Cuantificada(Qt, Initiates(0, 0, 0, e=e, f=f), sit)
print(form1)

((Initiates(e_0,f_0,0)∧Initiates(e_0,f_0,1))∧Initiates(e_0,f_0,2))


**Cuantificador existencial:**

In [14]:
e = sit.eventos[0]
f = sit.fluentes[0]
Et = Cuantificador('instante', 'existe', 0)
form2 = Cuantificada(Et, Initiates(0, 0, 0, e=e, f=f), sit)
print(form2)

((Initiates(e_0,f_0,0)∨Initiates(e_0,f_0,1))∨Initiates(e_0,f_0,2))


In [15]:
sit.instantes

In [16]:
t0 = sit.instantes[0]
f6 = Antes(0, 1, t1=t0)
print(f6)

0<ti1


In [18]:
qt = Cuantificador('instante', 'todo', 1)
f7 = Cuantificada(qt, f6, sit)
print(f7)

((0<0∧0<1)∧0<2)


<a class="anchor" id="ej2"></a>**Ejercicio 2:** 

([Anterior ejercicio](#ej1)) ([Próximo ejercicio](#ej3))

Cree la fórmula `StoppedIn(f_0,0,2)`:

$$ \exists e,t \left(Happens(e,t)\land 0<t<2\land Terminates(e,f_0,t)\right) $$

**Nota:**

La respuesta debe ser:
```
((((((Happens(e_0,0)∧0<0)∧0<2)∧Terminates(e_0,f_0,0))∧(((Happens(e_0,1)∧0<1)∧1<2)∧Terminates(e_0,f_0,1)))∧(((Happens(e_0,2)∧0<2)∧2<2)∧Terminates(e_0,f_0,2)))∧(((((Happens(e_1,0)∧0<0)∧0<2)∧Terminates(e_1,f_0,0))∧(((Happens(e_1,1)∧0<1)∧1<2)∧Terminates(e_1,f_0,1)))∧(((Happens(e_1,2)∧0<2)∧2<2)∧Terminates(e_1,f_0,2))))
```

In [27]:
t0 = sit.instantes[0]
t2 = sit.instantes[2]
h1 = Happens(0,1)
print(h1)

a1 = Antes(0, 1, t1 = t0)
print(a1)

a2 = Antes(1, 2, t2 = t2)
print(a2)

f1 = sit.fluentes[0]
term1 = Terminates(0, 0, 1, f = f1)
print(term1)

y1 = Y([h1, a1, a2, term1])
print(y1)

qt = Cuantificador('instante', 'existe', 1)
formula = Cuantificada(qt, y1, sit)
print(formula)

qt2 = Cuantificador('evento', 'existe', 0)
formula2 = Cuantificada(qt2, formula, sit)
print(formula2)

Happens(ev0,ti1)
0<ti1
ti1<2
Terminates(ev0,f_0,ti1)
(((Happens(ev0,ti1)∧0<ti1)∧ti1<2)∧Terminates(ev0,f_0,ti1))
(((((Happens(ev0,0)∧0<0)∧0<2)∧Terminates(ev0,f_0,0))∨(((Happens(ev0,1)∧0<1)∧1<2)∧Terminates(ev0,f_0,1)))∨(((Happens(ev0,2)∧0<2)∧2<2)∧Terminates(ev0,f_0,2)))
((((((Happens(e_0,0)∧0<0)∧0<2)∧Terminates(e_0,f_0,0))∨(((Happens(e_0,1)∧0<1)∧1<2)∧Terminates(e_0,f_0,1)))∨(((Happens(e_0,2)∧0<2)∧2<2)∧Terminates(e_0,f_0,2)))∨(((((Happens(e_1,0)∧0<0)∧0<2)∧Terminates(e_1,f_0,0))∨(((Happens(e_1,1)∧0<1)∧1<2)∧Terminates(e_1,f_0,1)))∨(((Happens(e_1,2)∧0<2)∧2<2)∧Terminates(e_1,f_0,2))))


---

# Axiomas <a class="anchor" id="axs"></a>

([Volver al inicio](#inicio))

$$ \left(Happens(e,t_1)\land Initiates(e,f,t_1)\land t_1<t_2\right)\to HoldsAt(f, t_2) $$

In [28]:
def axioma_lite_initiates(e, f):
    ante = Y([
        Happens(0, 1), 
        Initiates(0, 0, 1),
        Antes(1, 2), 
    ])
    cons = HoldsAt(0, 2)
    reg = Regla(ante, cons)
    Ee = Cuantificador('evento', 'todo', 0)
    axioma = Cuantificada(Ee, reg, sit)
    Ef = Cuantificador('fluente', 'todo', 0)
    axioma = Cuantificada(Ef, axioma, sit)
    Et1 = Cuantificador('instante', 'todo', 1)
    axioma = Cuantificada(Et1, axioma, sit)
    Et2 = Cuantificador('instante', 'todo', 2)
    return Cuantificada(Et2, axioma, sit)

e = sit.eventos[0]
f = sit.fluentes[0]
axio_lite1 = axioma_lite_initiates(e, f)
print(axio_lite1)

((((((((Happens(e_0,0)∧Initiates(e_0,f_0,0))∧0<0)⇒HoldsAt(f_0,0))∧(((Happens(e_1,0)∧Initiates(e_1,f_0,0))∧0<0)⇒HoldsAt(f_0,0)))∧((((Happens(e_0,1)∧Initiates(e_0,f_0,1))∧1<0)⇒HoldsAt(f_0,0))∧(((Happens(e_1,1)∧Initiates(e_1,f_0,1))∧1<0)⇒HoldsAt(f_0,0))))∧((((Happens(e_0,2)∧Initiates(e_0,f_0,2))∧2<0)⇒HoldsAt(f_0,0))∧(((Happens(e_1,2)∧Initiates(e_1,f_0,2))∧2<0)⇒HoldsAt(f_0,0))))∧((((((Happens(e_0,0)∧Initiates(e_0,f_0,0))∧0<1)⇒HoldsAt(f_0,1))∧(((Happens(e_1,0)∧Initiates(e_1,f_0,0))∧0<1)⇒HoldsAt(f_0,1)))∧((((Happens(e_0,1)∧Initiates(e_0,f_0,1))∧1<1)⇒HoldsAt(f_0,1))∧(((Happens(e_1,1)∧Initiates(e_1,f_0,1))∧1<1)⇒HoldsAt(f_0,1))))∧((((Happens(e_0,2)∧Initiates(e_0,f_0,2))∧2<1)⇒HoldsAt(f_0,1))∧(((Happens(e_1,2)∧Initiates(e_1,f_0,2))∧2<1)⇒HoldsAt(f_0,1)))))∧((((((Happens(e_0,0)∧Initiates(e_0,f_0,0))∧0<2)⇒HoldsAt(f_0,2))∧(((Happens(e_1,0)∧Initiates(e_1,f_0,0))∧0<2)⇒HoldsAt(f_0,2)))∧((((Happens(e_0,1)∧Initiates(e_0,f_0,1))∧1<2)⇒HoldsAt(f_0,2))∧(((Happens(e_1,1)∧Initiates(e_1,f_0,1))∧1<2)⇒HoldsAt(f_0,

<a class="anchor" id="ej3"></a>**Ejercicio 3:** 

([Anterior ejercicio](#ej2)) ([Próximo ejercicio](#ej4))

Cree el axioma para controlar que un evento termina un fluente:

$$ \left(Happens(e,t_1)\land Terminates(e,f,t_1)\land t_1<t_2\right)\to\neg HoldsAt(f, t_2) $$

**Nota:**

La respuesta debe ser

```
((((((((Happens(e_0,0)∧Terminates(e_0,f_0,0))∧0<0)⇒-HoldsAt(f_0,0))∧(((Happens(e_1,0)∧Terminates(e_1,f_0,0))∧0<0)⇒-HoldsAt(f_0,0)))∧((((Happens(e_0,1)∧Terminates(e_0,f_0,1))∧1<0)⇒-HoldsAt(f_0,0))∧(((Happens(e_1,1)∧Terminates(e_1,f_0,1))∧1<0)⇒-HoldsAt(f_0,0))))∧((((Happens(e_0,2)∧Terminates(e_0,f_0,2))∧2<0)⇒-HoldsAt(f_0,0))∧(((Happens(e_1,2)∧Terminates(e_1,f_0,2))∧2<0)⇒-HoldsAt(f_0,0))))∧((((((Happens(e_0,0)∧Terminates(e_0,f_0,0))∧0<1)⇒-HoldsAt(f_0,1))∧(((Happens(e_1,0)∧Terminates(e_1,f_0,0))∧0<1)⇒-HoldsAt(f_0,1)))∧((((Happens(e_0,1)∧Terminates(e_0,f_0,1))∧1<1)⇒-HoldsAt(f_0,1))∧(((Happens(e_1,1)∧Terminates(e_1,f_0,1))∧1<1)⇒-HoldsAt(f_0,1))))∧((((Happens(e_0,2)∧Terminates(e_0,f_0,2))∧2<1)⇒-HoldsAt(f_0,1))∧(((Happens(e_1,2)∧Terminates(e_1,f_0,2))∧2<1)⇒-HoldsAt(f_0,1)))))∧((((((Happens(e_0,0)∧Terminates(e_0,f_0,0))∧0<2)⇒-HoldsAt(f_0,2))∧(((Happens(e_1,0)∧Terminates(e_1,f_0,0))∧0<2)⇒-HoldsAt(f_0,2)))∧((((Happens(e_0,1)∧Terminates(e_0,f_0,1))∧1<2)⇒-HoldsAt(f_0,2))∧(((Happens(e_1,1)∧Terminates(e_1,f_0,1))∧1<2)⇒-HoldsAt(f_0,2))))∧((((Happens(e_0,2)∧Terminates(e_0,f_0,2))∧2<2)⇒-HoldsAt(f_0,2))∧(((Happens(e_1,2)∧Terminates(e_1,f_0,2))∧2<2)⇒-HoldsAt(f_0,2)))))
```

In [ ]:
def axioma_lite_terminates(e, f):
    pass
    ante = Y([
        Happens(0, 1), 
        Initiates(0, 0, 1),
        Antes(1, 2), 
    ])
    cons = HoldsAt(0, 2)
    reg = Regla(ante, cons)
    Ee = Cuantificador('evento', 'todo', 0)
    axioma = Cuantificada(Ee, reg, sit)
    Ef = Cuantificador('fluente', 'todo', 0)
    axioma = Cuantificada(Ef, axioma, sit)
    Et1 = Cuantificador('instante', 'todo', 1)
    axioma = Cuantificada(Et1, axioma, sit)
    Et2 = Cuantificador('instante', 'todo', 2)
    return Cuantificada(Et2, axioma, sit)


e = sit.eventos[1]
f = sit.fluentes[0]
axio_lite2 = axioma_lite_terminates(e, f)
print(axio_lite2)      

---

# Uso de bases de conocimiento <a class="anchor" id="base"></a>

([Volver al inicio](#inicio))

In [29]:
def limpiar_para_reglas(formula):
    reglas = str(formula).split(')∧(')
    #reglas = [r[1:] for r in reglas]
    reglas = [r.replace(')⇒', '⇒') for r in reglas]
    reglas = [r.replace('((', '') for r in reglas]
    reglas = [r.replace('))', ')') for r in reglas]
    reglas = [r.replace('))', ')') for r in reglas]
    reglas = [r.replace('(H', 'H') for r in reglas]
    return reglas

reglas = limpiar_para_reglas(axio_lite1)
reglas

['Happens(e_0,0)∧Initiates(e_0,f_0,0)∧0<0⇒HoldsAt(f_0,0)',
 'Happens(e_1,0)∧Initiates(e_1,f_0,0)∧0<0⇒HoldsAt(f_0,0)',
 'Happens(e_0,1)∧Initiates(e_0,f_0,1)∧1<0⇒HoldsAt(f_0,0)',
 'Happens(e_1,1)∧Initiates(e_1,f_0,1)∧1<0⇒HoldsAt(f_0,0)',
 'Happens(e_0,2)∧Initiates(e_0,f_0,2)∧2<0⇒HoldsAt(f_0,0)',
 'Happens(e_1,2)∧Initiates(e_1,f_0,2)∧2<0⇒HoldsAt(f_0,0)',
 'Happens(e_0,0)∧Initiates(e_0,f_0,0)∧0<1⇒HoldsAt(f_0,1)',
 'Happens(e_1,0)∧Initiates(e_1,f_0,0)∧0<1⇒HoldsAt(f_0,1)',
 'Happens(e_0,1)∧Initiates(e_0,f_0,1)∧1<1⇒HoldsAt(f_0,1)',
 'Happens(e_1,1)∧Initiates(e_1,f_0,1)∧1<1⇒HoldsAt(f_0,1)',
 'Happens(e_0,2)∧Initiates(e_0,f_0,2)∧2<1⇒HoldsAt(f_0,1)',
 'Happens(e_1,2)∧Initiates(e_1,f_0,2)∧2<1⇒HoldsAt(f_0,1)',
 'Happens(e_0,0)∧Initiates(e_0,f_0,0)∧0<2⇒HoldsAt(f_0,2)',
 'Happens(e_1,0)∧Initiates(e_1,f_0,0)∧0<2⇒HoldsAt(f_0,2)',
 'Happens(e_0,1)∧Initiates(e_0,f_0,1)∧1<2⇒HoldsAt(f_0,2)',
 'Happens(e_1,1)∧Initiates(e_1,f_0,1)∧1<2⇒HoldsAt(f_0,2)',
 'Happens(e_0,2)∧Initiates(e_0,f_0,2)∧2<2⇒HoldsAt(f_0,2)

In [30]:
b = LPQuery(reglas)
print(b)

Hechos:

Reglas:
Happens(e_0,0)∧Initiates(e_0,f_0,0)∧0<0⇒HoldsAt(f_0,0)
Happens(e_1,0)∧Initiates(e_1,f_0,0)∧0<0⇒HoldsAt(f_0,0)
Happens(e_0,1)∧Initiates(e_0,f_0,1)∧1<0⇒HoldsAt(f_0,0)
Happens(e_1,1)∧Initiates(e_1,f_0,1)∧1<0⇒HoldsAt(f_0,0)
Happens(e_0,2)∧Initiates(e_0,f_0,2)∧2<0⇒HoldsAt(f_0,0)
Happens(e_1,2)∧Initiates(e_1,f_0,2)∧2<0⇒HoldsAt(f_0,0)
Happens(e_0,0)∧Initiates(e_0,f_0,0)∧0<1⇒HoldsAt(f_0,1)
Happens(e_1,0)∧Initiates(e_1,f_0,0)∧0<1⇒HoldsAt(f_0,1)
Happens(e_0,1)∧Initiates(e_0,f_0,1)∧1<1⇒HoldsAt(f_0,1)
Happens(e_1,1)∧Initiates(e_1,f_0,1)∧1<1⇒HoldsAt(f_0,1)
Happens(e_0,2)∧Initiates(e_0,f_0,2)∧2<1⇒HoldsAt(f_0,1)
Happens(e_1,2)∧Initiates(e_1,f_0,2)∧2<1⇒HoldsAt(f_0,1)
Happens(e_0,0)∧Initiates(e_0,f_0,0)∧0<2⇒HoldsAt(f_0,2)
Happens(e_1,0)∧Initiates(e_1,f_0,0)∧0<2⇒HoldsAt(f_0,2)
Happens(e_0,1)∧Initiates(e_0,f_0,1)∧1<2⇒HoldsAt(f_0,2)
Happens(e_1,1)∧Initiates(e_1,f_0,1)∧1<2⇒HoldsAt(f_0,2)
Happens(e_0,2)∧Initiates(e_0,f_0,2)∧2<2⇒HoldsAt(f_0,2)
Happens(e_1,2)∧Initiates(e_1,f_0,2)∧2<2⇒HoldsAt(

In [31]:
tps = [t.valor for t in sit.instantes]
hechos_tiempos = [f'{x}<{y}' for x in tps for y in tps if x<y]
hechos_tiempos

['0<1', '0<2', '1<2']

In [32]:
hechos = ['HoldsAt(f_0,0)', 'Happens(e_0,0)']
hechos += [f'Initiates(e_0,f_0,{t})' for t in tps]
hechos += hechos_tiempos
for h in hechos:
    b.TELL(h)
print(b)

Hechos:
HoldsAt(f_0,0)
Happens(e_0,0)
Initiates(e_0,f_0,0)
Initiates(e_0,f_0,1)
Initiates(e_0,f_0,2)
0<1
0<2
1<2

Reglas:
Happens(e_0,0)∧Initiates(e_0,f_0,0)∧0<0⇒HoldsAt(f_0,0)
Happens(e_1,0)∧Initiates(e_1,f_0,0)∧0<0⇒HoldsAt(f_0,0)
Happens(e_0,1)∧Initiates(e_0,f_0,1)∧1<0⇒HoldsAt(f_0,0)
Happens(e_1,1)∧Initiates(e_1,f_0,1)∧1<0⇒HoldsAt(f_0,0)
Happens(e_0,2)∧Initiates(e_0,f_0,2)∧2<0⇒HoldsAt(f_0,0)
Happens(e_1,2)∧Initiates(e_1,f_0,2)∧2<0⇒HoldsAt(f_0,0)
Happens(e_0,0)∧Initiates(e_0,f_0,0)∧0<1⇒HoldsAt(f_0,1)
Happens(e_1,0)∧Initiates(e_1,f_0,0)∧0<1⇒HoldsAt(f_0,1)
Happens(e_0,1)∧Initiates(e_0,f_0,1)∧1<1⇒HoldsAt(f_0,1)
Happens(e_1,1)∧Initiates(e_1,f_0,1)∧1<1⇒HoldsAt(f_0,1)
Happens(e_0,2)∧Initiates(e_0,f_0,2)∧2<1⇒HoldsAt(f_0,1)
Happens(e_1,2)∧Initiates(e_1,f_0,2)∧2<1⇒HoldsAt(f_0,1)
Happens(e_0,0)∧Initiates(e_0,f_0,0)∧0<2⇒HoldsAt(f_0,2)
Happens(e_1,0)∧Initiates(e_1,f_0,0)∧0<2⇒HoldsAt(f_0,2)
Happens(e_0,1)∧Initiates(e_0,f_0,1)∧1<2⇒HoldsAt(f_0,2)
Happens(e_1,1)∧Initiates(e_1,f_0,1)∧1<2⇒HoldsAt(f_0,2

In [33]:
objetivo = 'HoldsAt(f_0,1)'
ASK(objetivo, 'success', b)

True

<a class="anchor" id="ej4"></a>**Ejercicio 4:** 

([Anterior ejercicio](#ej3)) ([Próximo ejercicio](#ej5))

Incluya en la base de conocimiento el hecho de que el evento `APAGAR` ocurre en el instante de tiempo 1. Incluya el axioma `axio_lite2` en la base y haga la consulta si `-HoldsAt(f_0, 2)`.

---

<a class="anchor" id="ej5"></a>**Ejercicio 5:** 

([Anterior ejercicio](#ej4)) ([Próximo ejercicio](#ej6))

Intuitivamente, la respuesta a la anterior consulta debe ser `True`, pero la base está incompleta. Falta la información de que el evento `APAGAR` termina el fluente `encendida`. Incluya esta información en la base de conocimiento y vuelva a hacer la consulta. Esta vez, la consulta debe arrojar `True`.

---

# Codificación de fórmulas fundamentadas <a class="anchor" id="cod"></a>

([Volver al inicio](#inicio))

Creamos un objeto descriptor y lo incluimos como un atributo de la situación.

**Nota:**

Es muy importante primero llamar el método `actualizar()` del objeto `sit` antes de crear el descriptor. Esto con el propósito de actualizar los predicados y los objetos de la situación. El número de estos será usado en la creación del descriptor.

In [34]:
sit.actualizar()
sit.crear_descriptor()
print("Predicados:\n", sit.predicados)
print('')
print("Eventos, fluentes, instantes, objetos:\n", sit.todos)

Predicados:
 ['Happens', 'HoldsAt', 'Initiates', 'Terminates', 'Antes', 'APAGAR', 'ENCENDER', 'Encendida']

Eventos, fluentes, instantes, objetos:
 ['e_0', 'e_1', 'f_0', '0', '1', '2', 'luz']


**Codificar:**

Todo objeto de tipo `Formula` tiene un método `codificar()`. El resultado es un caracter único mediante el cual se codifica la fórmula. Esto no lo podemos leer los humanos, pero será muy importante en la utilización de la transformación de Tseitin.

In [ ]:
A = haps.codificar(sit)
print(haps, '=>', A)
A = term2.codificar(sit)
print(term2, '=>', A)
ato = sit.fluentes[0].formula
A = ato.codificar(sit)
print(ato, '=>', A)

**Decodificar:**

El objeto `sit` tiene un método `formular()` que es el inverso de `codificar()` en las fórmulas. El metodo recibe una fórmula codificada y devuelve un objeto `Formula` de la sublase correspondiente:

In [ ]:
A = haps.codificar(sit)
print(haps, '=>', A, '=>', sit.formular(A))
A = term2.codificar(sit)
print(term2, '=>', A, '=>', sit.formular(A))
ato = sit.fluentes[0].formula
A = ato.codificar(sit)
print(ato, '=>', A, '=>', sit.formular(A))

<a class="anchor" id="ej6"></a>**Ejercicio 6:** 

([Anterior ejercicio](#ej5)) ([Próximo ejercicio](#ej7))

Encuentre la codificación de la fórmula:

```
(Happens(e_0,1)∧(Happens(e_0,1)⇒Terminates(e_0,f_0,0)))
```

**Nota:**

La respuesta debe ser

```
(Ǡ∧(Ǡ⇒؋))
```

Observe que la codificación de una fórmula compuesta no es un caracter, sino una fórmula cuyos átomos son caracteres.

---

# Uso de DPLL <a class="anchor" id="dpll"></a>

([Volver al inicio](#inicio))

Cuando ya tenemos una fórmula codificada, podemos encontrar su transformación de Tseitin:

In [ ]:
B = tseitin(A)
print(B)

Observe que hemos transformado la fórmula a su forma clausal. Adicionalmente, no tenemos caracteres, sino números. Para ver qué representa cada número, podemos usar el siguiente código:

In [ ]:
n, l = max_letras(A)
B = tseitin(A)
C = list(set([abs(val) for sublist in B for val in sublist]))
[f'{x} => {str(sit.formular(chr(x+256)))}' for x in C if x<n]

Los números que aparecen en la transformación que no se decodificaron son los átomos nuevos incluidos en el proceso de transformación de Tseitin.

Podemos ahora usar el `solve` de la libería `pycosat` para encontrar un modelo para la fórmula:

In [ ]:
res = pycosat.solve(B)
[str(sit.formular(chr(x+256))) for x in res if 0<x<n]

Si `B` es satisfacible, entonces `solve` encontrará un modelo. Si no, entonces retornará `UNSAT`.

<a class="anchor" id="ej7"></a>**Ejercicio 7:** 

([Anterior ejercicio](#ej6)) ([Próximo ejercicio](#ej8))

Use el `solve` para determinar si la fórmula `(Happens(e_0,1)∧(Happens(e_0,1)⇒Terminates(e_0,f_0,0)))` implica lógicamente a `Terminates(e_0,f_0,0)`.

Use los siguientes resultados de su curso de lógica 2:

* Si $U=\{A_1,\ldots, A_n\}$ es un conjunto de fórmulas y $B$ una fórmula, entonces $U\models B$ sii $(A_1\land\ldots\land A_n)\to B$ es una fórmula válida.

* Si $A$ es una fórmula, entonces $A$ es válida sii $\neg A$ es insatisfacible.

In [ ]:
def ASK_dpll(objetivo, premisas, valor, sit):
    # AQUI LA FÓRMULA -A
    # AQUI LA CODIFICACION DE -A
    # AQUI LA TRANSFORMADA DE TSEITIN, B
    res = pycosat.solve(B)
    return ((res == 'UNSAT') and valor == 'success')

ASK_dpll(term2, Y([haps,Regla(haps, term2)]), 'success', sit)

---

Mediante el DPLL podemos replicar el resultado anterior en el que usábamos la base de conocimiento. Para ello, sólo necesitamos una  fórmula. Esta fórmula será bastante extensa, compuesta de varias partes. Para las premisas, en primer lugar, incluiremos el axioma :

In [ ]:
lista = []
lista.append(axio_lite1)
[str(f) for f in lista]

A continuación incluimos la información lineal de los instantes:

In [ ]:
hechos_tiempos = [Antes(x.valor, y.valor, x, y) for x in sit.instantes for y in sit.instantes if x.valor<y.valor]
hechos_tiempos += [Negacion(Antes(x.valor, y.valor, x, y)) for x in sit.instantes for y in sit.instantes if not x.valor<y.valor]
lista += hechos_tiempos
[str(f) for f in lista]

Finalmente, incluimos los hechos:

In [ ]:
e = sit.eventos[0]
f = sit.fluentes[0]
t0 = sit.instantes[0]
t1 = sit.instantes[1]
lista.append(Happens(0,0,e,t0))
for t in sit.instantes:
    lista.append(Initiates(0, 0, 0, e=e, f=f, t=t))
premisas = Y(lista)
print(premisas)

Verificamos que la codificación haya sido satisfactoria:

In [ ]:
n, atomos = max_letras(premisas.codificar(sit))
atomos.sort()
[f'{x} => {str(sit.formular(chr(x+256)))}' for x in atomos if x<n]

Verificamos que no haya errores en la transformación de Tseitin:

In [ ]:
n, l = max_letras(premisas.codificar(sit))
B = tseitin(premisas.codificar(sit))
C = list(set([abs(val) for sublist in B for val in sublist]))
C.sort()
problemas = [x for x in C if (x not in atomos and x < n)]
if len(problemas) == 0:
    for x in C:
        if x<n:
            print([f'{x} => {str(sit.formular(chr(x+256)))}'])
else:
    print('¡Oh, Oh, problemas!', problemas)

Ahora creamos el objetivo

In [ ]:
objetivo = HoldsAt(0,0, f, t1)
print(objetivo)

Determinamos si el objetivo se sigue de las premisas:

In [ ]:
ASK_dpll(objetivo, premisas, 'success', sit)

<a class="anchor" id="ej8"></a>**Ejercicio 8:** 

([Anterior ejercicio](#ej7)) 

Cree la siguiente fórmula y use la función `ASK_dpll` para determinar si `HoldsAt(f_0,1)`:

$$ \left(Happens(e_1,t_1)\land Initiates(e,f,t_1)\land t_1<t_2\land \neg StoppedIn(t_1,f,t_2)\right)\to HoldsAt(f, t_2) $$

---

# Ejercicio

Represente la situación del ejemplo 2 de las diapositivas, sobre los teléfonos. Suponga que María siempre contesta cuando su teléfono suena. También suponga que Pedro llama a María. Determine si el teléfono de María está ocupado.

---

## En este notebook usted aprendió

* Implementar objetos, fórmulas y situaciones del Cálculo de Eventos en Python.
* Representar algunas situaciones sencillas.
* Usar una base de conocimiento para determinar si algunos fluentes son verdaderos.
* Usar un SAT solver para determinar si algunos fluentes son verdaderos.